# IMPORTS

## ALPHAPOSE IMPORTS

In [1]:
import argparse
import os
import platform
import sys
import time
import pickle as pkl

import numpy as np
import torch
from tqdm import tqdm
import natsort
import matplotlib
matplotlib.use('TkAgg')
from PIL import Image


alphapose_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose"

if alphapose_path  not in sys.path:
    sys.path.append(alphapose_path)

from detector.apis import get_detector
from trackers.tracker_api import Tracker
from trackers.tracker_cfg import cfg as tcfg
from trackers import track
from alphapose.models import builder
from alphapose.utils.config import update_config
from alphapose.utils.detector import DetectionLoader
from alphapose.utils.file_detector import FileDetectionLoader
from alphapose.utils.transforms import flip, flip_heatmap
from alphapose.utils.vis import getTime
from alphapose.utils.webcam_detector import WebCamDetectionLoader
from alphapose.utils.writer import DataWriter

## MOTIONBERT IMPORTS

In [2]:
motionbert_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBERT\MotionBERT"

if motionbert_path  not in sys.path:
    sys.path.append(motionbert_path)


import os
import numpy as np
import argparse
from tqdm import tqdm
import imageio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lib.utils.tools import *
from lib.utils.learning import *
from lib.utils.utils_data import flip_data
from lib.data.dataset_wild import WildDetDataset
from lib.utils.vismo import render_and_save
import os
import glob

In [3]:
import os
import random
import copy
import time
import sys
import shutil
import argparse
import errno
import math
import numpy as np
from collections import defaultdict, OrderedDict
import tensorboardX
from tqdm import tqdm
import pickle as pkl

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

from lib.utils.tools import *
from lib.model.loss import *
from lib.model.loss_mesh import *
from lib.utils.utils_mesh import *
from lib.utils.utils_smpl import *
from lib.utils.utils_data import *
from lib.utils.learning import *
from lib.data.dataset_mesh import MotionSMPL
from lib.model.model_mesh import MeshRegressor
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

import torch
import numpy as np
import glob
import os
import io
import random
import pickle
from torch.utils.data import Dataset, DataLoader
from lib.data.augmentation import Augmenter3D
from lib.utils.tools import read_pkl
from lib.utils.utils_data import flip_data, crop_scale
from lib.utils.utils_mesh import flip_thetas
from lib.utils.utils_smpl import SMPL
from torch.utils.data import Dataset, DataLoader
from lib.data.datareader_h36m import DataReaderH36M  
from lib.data.datareader_mesh import DataReaderMesh  
from lib.data.dataset_action import random_move  

In [4]:
DATASET_TYPE = 'train'

# EXTRACT ALPHAPOSE COORDS

In [5]:
class Args:
    def __init__(self):
        # Demo options
        self.config = 'config.yaml'  # experiment configure file name
        self.checkpoint = 'model.pth'  # checkpoint file name
        self.sp = False  # Use single process for pytorch
        self.detector = "yolo"  # detector name
        self.detfile = ""  # detection result file
        self.inputpath = "input_images"  # image-directory
        self.inputlist = ""  # image-list
        self.inputimg = "example.jpg"  # image-name
        self.outputpath = "examples/res/"  # output-directory
        self.save_img = False  # save result as image
        self.vis = False  # visualize image
        self.showbox = False  # visualize human bbox
        self.profile = False  # add speed profiling at screen output
        self.format = None  # save in format: coco/cmu/open
        self.min_box_area = 0  # min box area to filter out
        self.detbatch = 5  # detection batch size PER GPU
        self.posebatch = 64  # pose estimation maximum batch size PER GPU
        self.eval = False  # save result json as coco format with image index
        self.gpus = "0"  # CUDA devices to use, or -1 for CPU
        self.qsize = 1024  # length of result buffer
        self.flip = False  # enable flip testing
        self.debug = False  # print detail information

        # Video options
        self.video = ""  # video-name
        self.webcam = -1  # webcam number
        self.save_video = False  # whether to save rendered video
        self.vis_fast = False  # use fast rendering

        # Tracking options
        self.pose_flow = False  # track humans in video with PoseFlow
        self.pose_track = False  # track humans in video with reid

args = Args()

args.cfg = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\configs\halpe_26\resnet\256x192_res50_lr1e-3_1x_jupyter.yaml'
args.checkpoint  = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\pretrained_models\halpe26_fast_res50_256x192.pth"
args.video = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\downtown_bus_00\downtown_bus_00.mp4"
args.outputpath = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00'
args.save_video = False
args.vis_fast = False

cfg = update_config(args.cfg)


In [6]:
def print_finish_info():
    print('===========================> Finish Model Running.')
    if (args.save_img or args.save_video) and not args.vis_fast:
        print('===========================> Rendering remaining images in the queue...')
        print('===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).')

In [7]:
if platform.system() == 'Windows':
    args.sp = True

args.gpus = [int(i) for i in args.gpus.split(',')] if torch.cuda.device_count() >= 1 else [-1]
args.device = torch.device("cuda:" + str(args.gpus[0]) if args.gpus[0] >= 0 else "cpu")
args.detbatch = args.detbatch * len(args.gpus)
args.posebatch = args.posebatch * len(args.gpus)
args.tracking = args.pose_track or args.pose_flow or args.detector=='tracker'

print("Predictions will be done on device:")
print(args.device)

if not args.sp:
    torch.multiprocessing.set_start_method('forkserver', force=True)
    torch.multiprocessing.set_sharing_strategy('file_system')

Predictions will be done on device:
cuda:0


In [8]:
import os

# Initialize lists for test, train, and validation folders
test_folders = []
train_folders = []
val_folders = []

# Base directory path
base_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\sequenceFiles\sequenceFiles"

# Subfolder names
subfolders = {
    "test": test_folders,
    "train": train_folders,
    "validation": val_folders
}

# Iterate over each subfolder
for folder_name, file_list in subfolders.items():
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.exists(folder_path):  # Check if the folder exists
        for file in os.listdir(folder_path):
            if file.endswith(".pkl"):
                file_list.append(os.path.splitext(file)[0])  # Add the filename without extension

# Print the lists
print("Test folders:", test_folders)
print("Train folders:", train_folders)
print("Validation folders:", val_folders)


Test folders: ['downtown_arguing_00', 'downtown_bar_00', 'downtown_bus_00', 'downtown_cafe_00', 'downtown_car_00', 'downtown_crossStreets_00', 'downtown_downstairs_00', 'downtown_enterShop_00', 'downtown_rampAndStairs_00', 'downtown_runForBus_00', 'downtown_runForBus_01', 'downtown_sitOnStairs_00', 'downtown_stairs_00', 'downtown_upstairs_00', 'downtown_walkBridge_01', 'downtown_walking_00', 'downtown_walkUphill_00', 'downtown_warmWelcome_00', 'downtown_weeklyMarket_00', 'downtown_windowShopping_00', 'flat_guitar_01', 'flat_packBags_00', 'office_phoneCall_00', 'outdoors_fencing_01']
Train folders: ['courtyard_arguing_00', 'courtyard_backpack_00', 'courtyard_basketball_00', 'courtyard_bodyScannerMotions_00', 'courtyard_box_00', 'courtyard_capoeira_00', 'courtyard_captureSelfies_00', 'courtyard_dancing_01', 'courtyard_giveDirections_00', 'courtyard_golf_00', 'courtyard_goodNews_00', 'courtyard_jacket_00', 'courtyard_laceShoe_00', 'courtyard_rangeOfMotions_00', 'courtyard_relaxOnBench_00'

In [9]:
input_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles"
output_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

# Step 1: Traverse directories and find all the .mp4 files
video_paths = glob.glob(os.path.join(input_base_path, "**", "*.mp4"), recursive=True)

In [10]:
if DATASET_TYPE == 'train':
    video_paths = [path for path in video_paths if path.split(os.sep)[-2] in train_folders]
    for path in video_paths:
        print(path.split(os.sep)[-2])
elif DATASET_TYPE == 'test':
    video_paths = [path for path in video_paths if path.split(os.sep)[-2] in test_folders]
elif DATASET_TYPE == 'val':
    video_paths = [path for path in video_paths if path.split(os.sep)[-2] not in train_folders and path.split(os.sep)[-2] not in test_folders]


courtyard_arguing_00
courtyard_backpack_00
courtyard_basketball_00
courtyard_bodyScannerMotions_00
courtyard_box_00
courtyard_capoeira_00
courtyard_captureSelfies_00
courtyard_dancing_01
courtyard_giveDirections_00
courtyard_golf_00
courtyard_goodNews_00
courtyard_jacket_00
courtyard_laceShoe_00
courtyard_rangeOfMotions_00
courtyard_relaxOnBench_00
courtyard_relaxOnBench_01
courtyard_shakeHands_00
courtyard_warmWelcome_00
outdoors_climbing_00
outdoors_climbing_01
outdoors_climbing_02
outdoors_freestyle_00
outdoors_slalom_00
outdoors_slalom_01


In [11]:
output_train_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set"
output_test_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
output_val_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set"

In [12]:
import os
import glob


def initialize_models():
    """Load YOLO and Pose models once."""
    print("Loading YOLO detector...")
    yolo_detector = get_detector(args)  # Replace with actual detector initialization
    print("Loading pose model...")
    pose_model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
    pose_model.load_state_dict(torch.load(args.checkpoint, map_location=args.device))
    if len(args.gpus) > 1:
        pose_model = torch.nn.DataParallel(pose_model, device_ids=args.gpus).to(args.device)
    else:
        pose_model.to(args.device)
    pose_model.eval()
    print("Models loaded successfully.")
    return yolo_detector, pose_model

i_ = []

if __name__ == "__main__":
    # Load models once
    yolo_detector, pose_model = initialize_models()

    for video_path in video_paths:
        
        # Generate the output directory structure
        relative_path = os.path.relpath(video_path, input_base_path)
        cur_folder = os.path.basename(os.path.dirname(video_path))
        if cur_folder in train_folders:
            output_base_path = output_train_base_path
        elif cur_folder in test_folders:
            output_base_path = output_test_base_path
        elif cur_folder in val_folders:
            output_base_path = output_val_base_path
        else:
            print(f'Skipping folder {cur_folder}, because its set is not defined.')
            continue

        output_dir = os.path.join(output_base_path, os.path.dirname(relative_path))
        os.makedirs(output_dir, exist_ok=True)
        
        # Set the output path for the specific video
        args.outputpath = output_dir
        input_source = video_path
        
        mode = "video"  # Assume videos for this example; adapt based on your needs
        print(f"Processing video: {video_path}")
        print(f'Saving in {output_base_path}')
        
        # Load detection loader for the specific video
        det_loader = DetectionLoader(input_source, yolo_detector, cfg, args, batchSize=args.detbatch, mode=mode, queueSize=args.qsize)
        det_worker = det_loader.start()
        
        runtime_profile = {
            'dt': [],
            'pt': [],
            'pn': []
        }
        
        # Init data writer
        queueSize = 2 if mode == 'webcam' else args.qsize
        if args.save_video and mode != 'image':
            from alphapose.utils.writer import DEFAULT_VIDEO_SAVE_OPT as video_save_opt
            video_save_opt['savepath'] = os.path.join(args.outputpath, 'AlphaPose_' + os.path.basename(input_source))
            video_save_opt.update(det_loader.videoinfo)
            writer = DataWriter(cfg, args, save_video=True, video_save_opt=video_save_opt, queueSize=queueSize).start()
        else:
            writer = DataWriter(cfg, args, save_video=False, queueSize=queueSize).start()
        
        data_len = det_loader.length
        im_names_desc = tqdm(range(data_len), dynamic_ncols=True)

        batchSize = args.posebatch
        if args.flip:
            batchSize = int(batchSize / 2)
        try:
            for i in im_names_desc:
                start_time = getTime()
                with torch.no_grad():
                    (inps, orig_img, im_name, boxes, scores, ids, cropped_boxes) = det_loader.read()
                    # # Create a zero-filled tensor for keypoints
                    # # Example of printing values for debugging
                    # print("Inputs:", inps.shape)                # Print inputs
                    # print("Original Image:", orig_img.shape)    # Print the original image
                    # print("Image Name:", im_name)          # Print the image name
                    # print("Bounding Boxes:", boxes.shape)       # Print bounding boxes
                    # print("Scores:", scores.shape)               # Print scores
                    # print("IDs:", ids.shape)                     # Print object IDs
                    # print("Cropped Boxes:", cropped_boxes.shape)  # Print cropped bounding boxes
                    if orig_img is None:
                        break
                    if boxes is None or boxes.nelement() == 0:
                        print('No boxes for frame', i)
                        # Create a zero-filled tensor for keypoints
                        # Example of printing values for debugging
                        # print("Inputs:", inps)                # Print inputs
                        # print("Original Image:", orig_img)    # Print the original image
                        # print("Image Name:", im_name)          # Print the image name
                        # print("Bounding Boxes:", boxes)       # Print bounding boxes
                        # print("Scores:", scores)               # Print scores
                        # print("IDs:", ids)                     # Print object IDs
                        # print("Cropped Boxes:", cropped_boxes)  # Print cropped bounding boxes
                        # inps = torch.zeros((1, 3, 256, 192))
                        # im_name = torch.zeros((1, 3, 256, 192))
                        # boxes = torch.zeros((1, 4))  # Shape: [1, keypoints, (x, y, confidence)]
                        # scores = torch.zeros((1, 1))
                        # ids = torch.zeros((1, 1))
                        # cropped_boxes = torch.zeros((1, 4))
                        # writer.save(boxes, scores, ids, boxes, cropped_boxes, orig_img, im_name)
                        prev_scores = torch.zeros_like(prev_scores)
                        writer.save(prev_boxes, prev_scores, prev_ids, prev_hm, prev_cropped_boxes, orig_img, im_name)
                        i_.append(i)
                        continue
                    if args.profile:
                        ckpt_time, det_time = getTime(start_time)
                        runtime_profile['dt'].append(det_time)
                    # Pose Estimation
                    inps = inps.to(args.device)
                    datalen = inps.size(0)
                    leftover = 0
                    if (datalen) % batchSize:
                        leftover = 1
                    num_batches = datalen // batchSize + leftover
                    hm = []
                    if i == 1276:
                        print("num batches", num_batches)
                    for j in range(num_batches):
                        inps_j = inps[j * batchSize:min((j + 1) * batchSize, datalen)]
                        if args.flip:
                            inps_j = torch.cat((inps_j, flip(inps_j)))
                        hm_j = pose_model(inps_j)
                        if args.flip:
                            hm_j_flip = flip_heatmap(hm_j[int(len(hm_j) / 2):], pose_dataset.joint_pairs, shift=True)
                            hm_j = (hm_j[0:int(len(hm_j) / 2)] + hm_j_flip) / 2
                        hm.append(hm_j)
                        i_.append(i)
                    hm = torch.cat(hm)
                    if len(hm) == 0:
                        print('length for', im_names_desc[i], 'is zero')
                    if args.profile:
                        ckpt_time, pose_time = getTime(ckpt_time)
                        runtime_profile['pt'].append(pose_time)
                    if args.pose_track:
                        boxes, scores, ids, hm, cropped_boxes = track(tracker, args, orig_img, inps, boxes, hm, cropped_boxes, im_name, scores)
                    # if i == 614:                  
                    #     print("Boxes:", boxes)
                    #     print("Scores:", scores)
                    #     print("IDs:", ids)
                    #     print("Heatmap:", hm)
                    #     print("Cropped Boxes:", cropped_boxes)
                    #     print("Original Image:", orig_img)
                    #     print("Image Name:", im_name)
                        
                    writer.save(boxes, scores, ids, hm, cropped_boxes, orig_img, im_name)

                    prev_boxes = boxes
                    prev_scores = scores
                    prev_ids = ids
                    prev_hm = hm
                    prev_cropped_boxes = cropped_boxes
                    if args.profile:
                        ckpt_time, post_time = getTime(ckpt_time)
                        runtime_profile['pn'].append(post_time)
                
                if args.profile:
                    # TQDM
                    im_names_desc.set_description(
                        'det time: {dt:.4f} | pose time: {pt:.4f} | post processing: {pn:.4f}'.format(
                            dt=np.mean(runtime_profile['dt']), pt=np.mean(runtime_profile['pt']), pn=np.mean(runtime_profile['pn']))
                    )
                    
            print_finish_info()
            while(writer.running()):
                time.sleep(1)
                print('===========================> Rendering remaining ' + str(writer.count()) + ' images in the queue...', end='\r')
            writer.stop()
            det_loader.stop()
        except Exception as e:
            print(repr(e))
            print('An error as above occurs when processing the images, please check it')
            pass
        except KeyboardInterrupt:
            print_finish_info()
            if args.sp:
                det_loader.terminate()
                while(writer.running()):
                    time.sleep(1)
                    print('===========================> Rendering remaining ' + str(writer.count()) + ' images in the queue...', end='\r')
                writer.stop()
            else:
                det_loader.terminate()
                writer.terminate()
                writer.clear_queues()
                det_loader.clear_queues()


Loading YOLO detector...
{'CONFIG': 'E:/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/detector/yolo/cfg/yolov3-spp.cfg', 'WEIGHTS': 'E:/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/detector/yolo/data/yolov3-spp.weights', 'INP_DIM': 608, 'NMS_THRES': 0.6, 'CONFIDENCE': 0.1, 'NUM_CLASSES': 80}
Loading pose model...


e:\miniconda-envs\main_pw3d\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
e:\miniconda-envs\main_pw3d\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Models loaded successfully.
Processing video: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\courtyard_arguing_00\courtyard_arguing_00.mp4
Saving in E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set


  0%|          | 0/765 [00:00<?, ?it/s]

Loading YOLO model..
E:/Users/Philipp/Dokumente/Pose_Estimation_3D/Alphapose/AlphaPose/detector/yolo/cfg/yolov3-spp.cfg


...

===========================> Finish Model Running.
Results have been written to json.
Processing video: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\outdoors_slalom_00\outdoors_slalom_00.mp4
Saving in E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set


100%|██████████| 333/333 [00:10<00:00, 31.65it/s]


===========================> Finish Model Running.
Results have been written to json.
Processing video: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\outdoors_slalom_01\outdoors_slalom_01.mp4
Saving in E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set


100%|██████████| 371/371 [00:11<00:00, 31.49it/s]


===========================> Finish Model Running.
Results have been written to json.


# SPLIT JSON FILES INTO PERSON 1 AND PERSON 2

In [13]:
threedpw_train = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\preprocessed_train\3dpw_train.npz", allow_pickle=True) # preprocessed by SPIN
threedpw_test = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\preprocessed_test\3dpw_test.npz", allow_pickle=True) # preprocessed by SPIN
threedpw_val = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\preprocessed_val\3dpw_validation.npz", allow_pickle=True) # preprocessed by SPIN

In [14]:
import json

In [15]:
with open(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\data\mesh\mesh_det_pw3d.pkl", 'rb') as f:
    dataset_pw3d = pkl.load(f)

In [16]:
def closest_keypoints_to_bbox(keypoint_sets, true_bbox):
    """
    Find the keypoint set whose minimal bounding box is closest to the true bounding box.

    Parameters:
        keypoint_sets (list): List of keypoint sets, where each set is in the format [x1, y1, c1, ...].
        true_bbox (tuple): Ground truth bounding box in the format (x_min, y_min, x_max, y_max).

    Returns:
        list: Keypoint set with the closest bounding box.
    """
    def calculate_bbox(keypoints):
        coords = [(keypoints[i], keypoints[i + 1]) for i in range(0, len(keypoints), 3)]
        x_coords, y_coords = zip(*coords)
        return min(x_coords), min(y_coords), max(x_coords), max(y_coords)

    def bbox_center(bbox):
        x_min, y_min, x_max, y_max = bbox
        return ((x_min + x_max) / 2, (y_min + y_max) / 2)

    true_center = bbox_center(true_bbox)
    
    closest_keypoints = None
    min_distance = float('inf')

    for keypoints in keypoint_sets:
        bbox = calculate_bbox(keypoints)
        center = bbox_center(bbox)
        distance = np.sqrt((center[0] - true_center[0])**2 + (center[1] - true_center[1])**2)

        if distance < min_distance:
            min_distance = distance
            closest_keypoints = keypoints

    return closest_keypoints

In [17]:
def calculate_min_bounding_box(keypoints):
    """
    Calculate the minimal bounding box that encapsulates the given keypoints.
    
    Parameters:
    - keypoints: A numpy array of shape (N, 2), where N is the number of keypoints.
    
    Returns:
    - bbox: A tuple containing the coordinates of the bounding box (min_x, min_y, max_x, max_y).
    """
    # Ensure keypoints is a numpy array
    import numpy as np
    keypoints = np.array(keypoints)
    
    # Get min and max for x and y coordinates
    min_x = np.min(keypoints[:, 0])
    min_y = np.min(keypoints[:, 1])
    max_x = np.max(keypoints[:, 0])
    max_y = np.max(keypoints[:, 1])
    
    # Bounding box coordinates: (min_x, min_y, max_x, max_y)
    bbox = (min_x, min_y, max_x, max_y)
    return bbox

In [18]:
def save_annot(filtered_data_p0, filtered_data_p1, folder_path):
    # Save filtered_data_p0 as 'person1.json', overwriting if it exists
    if filtered_data_p0 is not None:
        person1_path = os.path.join(folder_path, 'person_1.json')
        with open(person1_path, 'w') as json_file:
            json.dump(filtered_data_p0, json_file, indent=4)
        print(f"Saved person1 data to {person1_path}")

    # Save filtered_data_p1 as 'person2.json', overwriting if it exists
    if filtered_data_p1 is not None:
        person2_path = os.path.join(folder_path, 'person_2.json')
        with open(person2_path, 'w') as json_file:
            json.dump(filtered_data_p1, json_file, indent=4)
        print(f"Saved person2 data to {person2_path}")


In [19]:
def filter_duplicates(data, true_bboxes):
    from collections import defaultdict

    # Group dictionaries by image_id
    grouped_data = defaultdict(list)
    for entry in data:
        grouped_data[entry['image_id']].append(entry)

    # Prepare the filtered data list
    filtered_data = []

    # Process each group
    for idx, (image_id, entries) in enumerate(grouped_data.items()):
        true_bbox = true_bboxes[idx]  # Get the corresponding true bbox for the frame
        
        if len(entries) == 1:
            # If there's only one entry for this image_id, add it directly
            filtered_data.append(entries[0])
        else:
            # Extract keypoints and apply closest_keypoints_to_bbox
            keypoint_sets = [entry['keypoints'] for entry in entries]
            closest_keypoints = closest_keypoints_to_bbox(keypoint_sets, true_bbox)

            # Find the entry with the matching keypoints and add it
            for entry in entries:
                if entry['keypoints'] == closest_keypoints:
                    filtered_data.append(entry)
                    break

    return filtered_data

In [20]:
def get_file_name(idx):

    string = dataset_pw3d['test']['source'][idx]

    # Person index
    last_digit = string[-1]

    # File index
    second_digit = string[-2]

    new_string = string[:-3] + f'0{second_digit}'

    return new_string

In [21]:
# file_names = []


# for idx, elem in enumerate(dataset_pw3d['test']['source']):
#     file_names.append(get_file_name(idx))

In [22]:
# unique_file_names = list(dict.fromkeys(file_names))
# switch = unique_file_names[15]
# unique_file_names[15] = unique_file_names[16]
# unique_file_names[16] = switch

In [23]:
# Specify the directory path
if DATASET_TYPE == 'train':
    dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set"
if DATASET_TYPE == 'test':
    dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
if DATASET_TYPE == 'val':
    dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set"

# Get a list of folder names in the specified directory
unique_file_names = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

In [24]:
all_bboxes = []

if DATASET_TYPE == 'test':
    cur_dataset = threedpw_test
elif DATASET_TYPE == 'train':
    cur_dataset = threedpw_train
elif DATASET_TYPE == 'val':
    cur_dataset = threedpw_val


for file_idx, file_data in enumerate(cur_dataset['all_poses_2d']):
    file_bboxes = []

    for person_idx, person_data in enumerate(file_data):  # Adjust based on structure
        person_boxes = []

        for frame in person_data:  # Iterate over frames
            poses_2d = frame.T
            bbox = calculate_min_bounding_box(poses_2d)
            
            # If bbox is None, replace it with (0, 0, 0, 0)
            if bbox is None:
                bbox = (0, 0, 0, 0)

            person_boxes.append(bbox)

        file_bboxes.append(person_boxes)

    all_bboxes.append(file_bboxes)

In [25]:
# FIX FOLDER INDEX
# ALWAYS SAVING BOTH PERSONS LEADS TO THE FACT THAT IF THERE IS ONLY ! PERSON; THE PREVIOUS SECOND PERSON IS SAVED

if DATASET_TYPE == 'train':
    root_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set"
elif DATASET_TYPE == 'test':
    root_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
elif DATASET_TYPE == 'val':
    root_dir = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set"


# Loop through subfolders
# Loop through subfolders
for root, dirs, files in os.walk(root_dir):
    # print(f"Current directory: {root}")
    # print(f"Subdirectories: {dirs}")
    # print(f"Files: {files}")
    filtered_data_p0 = None
    filtered_data_p1 = None
    current_path = os.path.basename(root)
    for file in files:
        filtered_data_p0 = None
        filtered_data_p1 = None
        file_path = os.path.join(root, file)
        print(f"Processing file: {file_path}")
        # Check if the file is a JSON file
        if file.endswith('alphapose-results.json'):
            json_path = os.path.join(root, file)
            try:
                # Open and load the JSON file
                with open(json_path, 'r') as json_file:
                    data = json.load(json_file)

                folder_index = unique_file_names.index(current_path)
                # Use folder_index to access corresponding bounding boxes
                filtered_data_p0 = filter_duplicates(data, all_bboxes[folder_index][0])
                if len(all_bboxes[folder_index]) >= 2:
                    filtered_data_p1 = filter_duplicates(data, all_bboxes[folder_index][1])

                # Uncomment and define save_annot to save the filtered data
                save_annot(filtered_data_p0, filtered_data_p1, os.path.join(root))


                filtered_data_p0 = None
                filtered_data_p1 = None
                print(f"Successfully processed and saved: {json_path}")

            except Exception as e:
                print(f"Error loading {json_path}: {e}")


Processing file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set\courtyard_arguing_00\alphapose-results.json
Saved person1 data to E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set\courtyard_arguing_00\person_1.json
Saved person2 data to E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set\courtyard_arguing_00\person_2.json
Successfully processed and saved: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set\courtyard_arguing_00\alphapose-results.json
Processing file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set\courtyard_arguing_00\AlphaPose_courtyard_arguing_00.mp4
Processing file: E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set\courtyard_backpack_00\alphapose-results.json
Saved person1 data to E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\

# MOTIONBERT DETECTIONS

In [26]:
class Args:
    def __init__(self):
        # Define all your arguments as class attributes with default values
        self.config = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\configs\pose3d\MB_ft_h36m_global_lite.yaml"  # Path to the config file
        self.evaluate = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\checkpoint\pose3d\FT_MB_lite_MB_ft_h36m_global_lite\best_epoch.bin"  # Checkpoint to evaluate
        self.json_path = None  # Alphapose detection result JSON path
        self.vid_path = None  # Video path
        self.out_path = None  # Output path
        self.pixel = False  # Align with pixel coordinates
        self.focus = None  # Target person ID
        self.clip_len = 243  # Clip length for network input


opts = Args()
args = get_config(opts.config)


In [27]:
model_backbone = load_backbone(args)
if torch.cuda.is_available():
    model_backbone = nn.DataParallel(model_backbone)
    model_backbone = model_backbone.cuda()

print('Loading checkpoint')
checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
model_pos = model_backbone
model_pos.eval()
pass

Loading checkpoint


In [28]:
def main(opts):

    testloader_params = {
        'batch_size': 1,
        'shuffle': False,
        'num_workers': 8,
        'pin_memory': True,
        'prefetch_factor': 4,
        'persistent_workers': True,
        'drop_last': False
    }

    vid = imageio.get_reader(opts.vid_path, 'ffmpeg')
    fps_in = vid.get_meta_data()['fps']
    vid_size = vid.get_meta_data()['size']
    os.makedirs(opts.out_path, exist_ok=True)

    if opts.pixel:
        wild_dataset = WildDetDataset(opts.json_path, clip_len=opts.clip_len, vid_size=vid_size, scale_range=None, focus=opts.focus)
    else:
        wild_dataset = WildDetDataset(opts.json_path, clip_len=opts.clip_len, scale_range=[1, 1], focus=opts.focus)

    test_loader = DataLoader(wild_dataset, **testloader_params)

    results_all = []
    with torch.no_grad():
        for batch_input in tqdm(test_loader):
            N, T = batch_input.shape[:2]
            if torch.cuda.is_available():
                batch_input = batch_input.cuda()
            if args.no_conf:
                batch_input = batch_input[:, :, :, :2]
            if args.flip:
                batch_input_flip = flip_data(batch_input)
                predicted_3d_pos_1 = model_pos(batch_input)
                predicted_3d_pos_flip = model_pos(batch_input_flip)
                predicted_3d_pos_2 = flip_data(predicted_3d_pos_flip)
                predicted_3d_pos = (predicted_3d_pos_1 + predicted_3d_pos_2) / 2.0
            else:
                predicted_3d_pos = model_pos(batch_input)
            if args.rootrel:
                predicted_3d_pos[:, :, 0, :] = 0
            else:
                predicted_3d_pos[:, 0, 0, 2] = 0
            if args.gt_2d:
                predicted_3d_pos[..., :2] = batch_input[..., :2]
            results_all.append(predicted_3d_pos.cpu().numpy())

    results_all = np.hstack(results_all)
    results_all = np.concatenate(results_all)
    #render_and_save(results_all, f'{opts.out_path}/X3D.mp4', keep_imgs=False, fps=fps_in)
    if opts.pixel:
        results_all = results_all * (min(vid_size) / 2.0)
        results_all[:, :, :2] = results_all[:, :, :2] + np.array(vid_size) / 2.0
    np.save(f'{opts.out_path}/X3D.npy', results_all)

In [29]:
if DATASET_TYPE == 'train':
    current_folders = train_folders
elif DATASET_TYPE == 'test':
    current_folders = test_folders
elif DATASET_TYPE == 'val':
    current_folders = val_folders


In [30]:
import os
import glob
import gc

# Paths
video_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles"

if DATASET_TYPE == 'train':
    json_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set"
    output_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set"

if DATASET_TYPE == 'test':
    json_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
    output_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

if DATASET_TYPE == 'val':
    json_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set"
    output_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set"

overwrite = True  # Set to True if you want to force overwriting existing files

# Iterate through each folder in the video base path
for folder_name in os.listdir(video_base_path):
    if folder_name in current_folders:
        folder_path = os.path.join(video_base_path, folder_name)

        if os.path.isdir(folder_path):
            video_file = os.path.join(folder_path, f"{folder_name}.mp4")

            if os.path.exists(video_file):
                json_folder = os.path.join(json_base_path, folder_name)

                if os.path.isdir(json_folder):
                    json_files = glob.glob(os.path.join(json_folder, "person_*.json"))

                    for json_file in json_files:
                        person_id = os.path.splitext(os.path.basename(json_file))[0].split('_')[-1]
                        output_path = os.path.join(output_base_path, folder_name, "MotionBert", f"person_{person_id}")

                        x3d_mp4 = os.path.join(output_path, "X3D.mp4")
                        x3d_npy = os.path.join(output_path, "X3D.npy")

                        if overwrite or not (os.path.exists(x3d_mp4) and os.path.exists(x3d_npy)):
                            print(f"Processing folder: {folder_name}, person: {person_id}")
                            
                            os.makedirs(output_path, exist_ok=True)

                            opts.vid_path = video_file
                            opts.json_path = json_file
                            opts.out_path = output_path
                            main(opts)

                            # Free up memory after processing
                        else:
                            print(f"Skipping {output_path}, output files already exist.")


Processing folder: courtyard_arguing_00, person: 1


100%|██████████| 4/4 [00:14<00:00,  3.60s/it]


Processing folder: courtyard_arguing_00, person: 2


100%|██████████| 4/4 [00:13<00:00,  3.39s/it]


Processing folder: courtyard_backpack_00, person: 1


...

100%|██████████| 3/3 [00:13<00:00,  4.34s/it]


Processing folder: outdoors_slalom_00, person: 1


100%|██████████| 2/2 [00:12<00:00,  6.31s/it]


Processing folder: outdoors_slalom_01, person: 1


100%|██████████| 2/2 [00:12<00:00,  6.32s/it]


# CALCULATING SMPL PARAMETERS

In [31]:
import sys

In [32]:
smpl_params_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Pose_to_SMPL\video2bvh"
if smpl_params_path not in sys.path:
    sys.path .append(smpl_params_path)



# from pose_estimator_2d import openpose_estimator
# from pose_estimator_3d import estimator_3d
from smpl_utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import cv2
import importlib
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML
import math
#import mathutils


In [33]:
mapping = [(0, None), (4, None), (1, None), (7, None), (5, None), (2, None), (3, None), (6, None), (12, None), (8, None), (15, None), (None, [0, 0, 0]), (9, None), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (10, None), (13, None), (11, None), (14, None), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0])]

In [34]:
h36m_skel = h36m_skeleton.H36mSkeleton()

In [35]:
from scipy.spatial.transform import Rotation as R

In [36]:
def euler_to_quaternion_zxy(euler_z, euler_x, euler_y):
    # Create a Rotation object with 'ZXY' order
    r = R.from_euler('zxy', [euler_z, euler_x, euler_y], degrees=True)

    # Convert to quaternion
    quaternion = r.as_quat()  # Returns (x, y, z, w)

    # Return as a tuple in the desired order (W, Y, Z, X)
    return (quaternion[3], quaternion[1], quaternion[2], quaternion[0])


def rodrigues_from_pose(frame, joint_name, all_rotations):
    # Extract quaternion from input dictionary
    quat = all_rotations[frame][joint_name]  # Assumes quaternion is in (W, X, Y, Z) order

    # Create a SciPy Rotation object
    rotation = R.from_quat([quat[1], quat[2], quat[3], quat[0]])  # Convert to (X, Y, Z, W)

    # Convert to axis-angle representation
    axis = rotation.as_rotvec()  # Returns Rodrigues vector directly (axis * angle)

    return axis


def apply_mapping(B, mapping, group_size=3):
    """
    Apply a mapping to reorder and pad a list like B.
    """
    # Divide B into groups of group_size
    grouped_B = [B[i:i + group_size] for i in range(0, len(B), group_size)]

    # Apply mapping
    output_groups = []
    for index, padding in mapping:
        if index is not None:
            output_groups.append(grouped_B[index])  # Reorder from B
        elif padding is not None:
            output_groups.append(padding)  # Add padding group

    # Flatten the output groups
    reordered_B = [item for group in output_groups for item in group]
    return reordered_B


def load_and_prepare_data(data):

    # Load the 3D pose data from the .npy file
    pose_data = data

    # Get the number of frames and joints
    n_frames, n_joints, _ = pose_data.shape

    # Define the rotation matrix for -90 degrees around the x-axis
    # rotation_matrix = np.array([
    #     [1,  0,  0],
    #     [0,  0, 1],
    #     [0,  -1,  0]
    # ])

    rotation_matrix = np.array([
        [1,  0,  0],
        [0,  0, 1],
        [0,  -1,  0]
    ])

    # Apply the rotation matrix to every frame and every joint
    # We reshape the pose_data to (n_frames * n_joints, 3), apply the rotation, then reshape back
    data = np.dot(pose_data.reshape(-1, 3), rotation_matrix.T).reshape(n_frames, n_joints, 3)


    return data

def calculate_translations(channels):

    all_translations = []

    for index in range(len(channels)):
        # Initialize the dictionary to hold the results
        # Iterate through the list and extract translation values
        translation_values = channels[index][0:3]  # Extracting the translation values

        all_translations.append(translation_values)

    return all_translations


def calculate_rotations(channels, body_parts):

    all_rotations = []

    for index in range(len(channels)):
        # Initialize the dictionary to hold the results
        rotation_dict = {}
        # Iterate through the list and extract rotation values
        rotation_values = channels[index][3:]  # Extracting the rotation values, skipping the first two
        # if index == 0:
        #     print(rotation_values)
        for i, body_part in enumerate(body_parts):
            # Take 3 consecutive values (x, y, z rotations) for each body part
            quat = euler_to_quaternion_zxy(-rotation_values[i*3+2], rotation_values[i*3+1], rotation_values[i*3])
            rotation_dict[body_part] = quat

        all_rotations.append(rotation_dict)

    return all_rotations

def calculate_all_poses(all_rotations, body_parts):

    joint_names = body_parts
    num_joints = len(joint_names)
    frame = 0
    all_poses = []


    for frame in range(len(all_rotations)):
        # Get armature pose in rodrigues representation
        pose = [0.0] * (num_joints * 3)

        for index in range(num_joints):
            joint_name = joint_names[index]
            joint_pose = rodrigues_from_pose(frame, joint_name, all_rotations)
            pose[index*3 + 0] = joint_pose[2]
            pose[index*3 + 1] = joint_pose[0]
            pose[index*3 + 2] = joint_pose[1]
        
        all_poses.append(pose)

    return all_poses

def calculate_all_thetas(all_poses):
    all_thetas = []

    for i, frame in enumerate(all_poses):
        theta_calculated = all_poses[i]

        reordered_theta = apply_mapping(theta_calculated, mapping)
        smpl_h_params = reordered_theta

        reordered_theta = reordered_theta[0:72]

        all_thetas.append(reordered_theta)
    return all_thetas

    #np.save("all_thetas_backpack.npy", all_thetas)

def save_smpl(translations, thetas, vertices, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    if translations is not None:
        np.save(os.path.join(output_folder, 'all_translations.npy'), translations)
    if thetas is not None:
        np.save(os.path.join(output_folder, 'all_thetas.npy'), thetas)
    if vertices is not None:
        np.save(os.path.join(output_folder, 'vertices.npy'), vertices)



In [37]:
def calculate_smpl_output(data):

    data = load_and_prepare_data(data)

    channels, header = h36m_skel.poses2bvh(data, output_file=None)

    all_translations = calculate_translations(channels)

    # List of body parts in the order matching the data
    body_parts = [
        'Hip', 'RightHip', 'RightKnee', 'RightAnkle', 'LeftHip', 'LeftKnee', 'LeftAnkle', 
        'Spine', 'Thorax', 'Neck', 'LeftShoulder', 'LeftElbow', 'LeftWrist', 
        'RightShoulder', 'RightElbow', 'RightWrist'
    ]

    all_rotations = calculate_rotations(channels, body_parts)

    all_poses = calculate_all_poses(all_rotations, body_parts)


    all_thetas = calculate_all_thetas(all_poses)

    return np.asarray(all_translations), np.asarray(all_thetas)

In [38]:
args.data_root = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\data\mesh'

In [39]:
smpl = SMPL(args.data_root, batch_size=1).double().cuda()

In [40]:
opts.vid_path = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\flat_guitar_01\flat_guitar_01.mp4'

In [41]:
vid = imageio.get_reader(opts.vid_path,  'ffmpeg')
fps_in = vid.get_meta_data()['fps']
vid_size = vid.get_meta_data()['size']

In [42]:
opts.out_path

'E:\\Users\\Philipp\\Dokumente\\Pose_Estimation_3D\\Alphapose\\AlphaPose\\examples\\res\\train_set\\outdoors_slalom_01\\MotionBert\\person_1'

In [43]:
# Base directory paths
dataset_dir = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW'

if DATASET_TYPE == 'train':
    base_pred_dir = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\train_set'
elif DATASET_TYPE == 'test':
    base_pred_dir = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set'
elif DATASET_TYPE == 'val':
    base_pred_dir = r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\val_set'

## FIRST WAY OF EXTRACTING VERTS

In [44]:
smpl_3dpw_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\readme"
if smpl_3dpw_path not in sys.path:
    sys.path .append(smpl_3dpw_path)

from smpl.smpl_webuser.serialization import load_model

In [45]:
motionbert_preds = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\X3D.npy")
predicted_translation, predicted_poses = calculate_smpl_output(motionbert_preds)

In [46]:
loaded_thetas = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\all_thetas.npy")

In [47]:
loaded_translation = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\all_translations.npy")

In [48]:
# Load a sequence file and its corresponding pose predictions
def process_sequence(seq_name):
    print(f"Processing sequence: {seq_name}")
    
    # Load sequence data
    seq_file = os.path.join(dataset_dir, 'sequenceFiles', 'sequenceFiles', f'{DATASET_TYPE}', seq_name + '.pkl')
    seq = pkl.load(open(seq_file, 'rb'), encoding='latin1')

    # Iterate over person folders in the sequence
    seq_pred_dir = os.path.join(base_pred_dir, seq_name, 'MotionBert')
    motionbert_preds = os.path.join(seq_pred_dir, 'X3D.npy')
    person_folders = [f for f in os.listdir(seq_pred_dir) if f.startswith("person_")]

    for person_folder in person_folders:
        person_number = int(person_folder[-1])
        person_path = os.path.join(seq_pred_dir, person_folder)
        motionbert_preds = np.load(os.path.join(seq_pred_dir, person_folder, 'X3D.npy'))
        predicted_translation, predicted_poses = calculate_smpl_output(motionbert_preds)

        print(f"Loaded poses for {person_folder}, shape: {predicted_poses.shape}")

        # Load the model 
        if person_number == 1:
            iModel = 0  # Assuming one model for simplicity
        elif person_number == 2:
            iModel = 1
        else: raise ValueError

        if seq['genders'][iModel] == 'm':
            model_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\readme\smpl\models\basicmodel_m_lbs_10_207_0_v1.0.0.pkl"
        else:
            model_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\readme\smpl\models\basicModel_f_lbs_10_207_0_v1.0.0.pkl"
        
        model = load_model(model_path)
        model.betas[:10] = np.zeros(10)  # Use zero betas or customize if needed

        valid_poses = []
        valid_transitions = []

        # Calculate vertices for all frames
        vertices_all = []
        for iFrame in range(len(seq['campose_valid'][iModel])):
            if seq['campose_valid'][iModel][iFrame]:
                valid_poses.append(predicted_poses[iFrame])
                valid_transitions.append(predicted_translation[iFrame])

                model.pose[:] = predicted_poses[iFrame]
                model.trans[:] = predicted_translation[iFrame]
                vertices = np.asarray(model)
                vertices_all.append(vertices)
        
        # calculate vertices

        print('valid poses shape', np.asarray(valid_poses).shape)

        # Save the vertices for the person
        vertices_all = np.array(vertices_all)  # Convert to numpy array
        # output_file = os.path.join(person_path, 'vertices.npy')
        # np.save(output_file, vertices_all)
        # print(f"Saved vertices to {output_file}, shape: {vertices_all.shape}")

        # all_thetas = predicted_poses[seq['campose_valid'][iModel]]
        # all_translation = predicted_translation[seq['campose_valid'][iModel]]

        # print('translation shape is', np.asarray(all_translation).shape)
        # Define the rotation matrix
        R_x = np.array([
            [1,  0,  0],
            [0, -1,  0],
            [0,  0, -1]
        ])

        # Assuming verts_all is already defined with shape (1262, 6890, 3)
        # Apply the rotation
        #vertices_all = np.einsum('ij,klj->kli', R_x, vertices_all)

        save_smpl(translations=predicted_translation, thetas=predicted_poses, vertices=np.asarray(vertices_all), output_folder = person_path)

# Main processing function
def process_all_sequences():
    # List all sequence folders
    sequence_folders = [f for f in os.listdir(base_pred_dir) if os.path.isdir(os.path.join(base_pred_dir, f))]
    
    for seq_name in sequence_folders:
        process_sequence(seq_name)

# Run the processing
process_all_sequences()

Processing sequence: courtyard_arguing_00
Loaded poses for person_1, shape: (765, 72)
valid poses shape (765, 72)
Loaded poses for person_2, shape: (765, 72)
valid poses shape (765, 72)
Processing sequence: courtyard_backpack_00
Loaded poses for person_1, shape: (1262, 72)
valid poses shape (1262, 72)
Processing sequence: courtyard_basketball_00
Loaded poses for person_1, shape: (468, 72)
valid poses shape (418, 72)
Loaded poses for person_2, shape: (468, 72)
valid poses shape (468, 72)
Processing sequence: courtyard_bodyScannerMotions_00
Loaded poses for person_1, shape: (1257, 72)
valid poses shape (1257, 72)
Processing sequence: courtyard_box_00
Loaded poses for person_1, shape: (1041, 72)
valid poses shape (1041, 72)
Processing sequence: courtyard_capoeira_00
Loaded poses for person_1, shape: (435, 72)
valid poses shape (388, 72)
Loaded poses for person_2, shape: (435, 72)
valid poses shape (435, 72)
Processing sequence: courtyard_captureSelfies_00
Loaded poses for person_1, shape:

In [ ]:
# EDIT SUCH THAT THETAS AND TRANSLATION ARE ALSO CALCULATED IN HERE

In [40]:
sequence_folders = [f for f in os.listdir(base_pred_dir) if os.path.isdir(os.path.join(base_pred_dir, f))]

In [ ]:
#function that takes the name of the sequence and the person number and returns the boolean array of compose valid

## SECOND WAY OF EXTRACTING VERTS

In [39]:
def get_campose_valid(seq_name, person_nr):
    seq_file = os.path.join(dataset_dir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
    seq = pkl.load(open(seq_file, 'rb'), encoding='latin1')
    
    
    # # Iterate over person folders in the sequence
    # seq_pred_dir = os.path.join(base_pred_dir, seq_name, 'MotionBert')
    # person_folders = [f for f in os.listdir(seq_pred_dir) if f.startswith("person_")]

    # for person_folder in person_folders:
    #     person_number = int(person_folder[-1])
    #     person_path = os.path.join(seq_pred_dir, person_folder)

    # Load the model (assuming only one model is needed per person)
    if person_nr == 1:
        iModel = 0  # Assuming one model for simplicity
    elif person_nr == 2:
        iModel = 1
    else: raise ValueError

    return seq['campose_valid'][iModel]

In [ ]:
# Take all motionbert files and calculate all SMPL parameters (+ vertices)

def process_motionbert_files(root_path):
    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if file == 'X3D.npy':
                # Full path to the data file
                data_path = os.path.join(dirpath, file)

                # Call calculate_smpl_output
                data = np.load(data_path)
                translations, thetas = calculate_smpl_output(data)


                thetas = torch.tensor(thetas).to('cuda')

                all_translations_tensor = torch.tensor(translations, dtype=torch.double).to('cuda')

                path_split_list = dirpath.split(os.sep)
                seq_name =  path_split_list[-3]
                person_nr = int(path_split_list[-1][-1]) 
                seq_file = os.path.join(dataset_dir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
                seq = pkl.load(open(seq_file, 'rb'), encoding='latin1')

                # campose_valid = get_campose_valid(seq_name, person_nr)
                # #print(len(campose_valid))

                # all_translations_tensor = all_translations_tensor[campose_valid]
                # thetas = thetas[campose_valid]


                seq_file = os.path.join(dataset_dir, 'sequenceFiles', 'sequenceFiles', 'test', seq_name + '.pkl')
                seq = pkl.load(open(seq_file, 'rb'), encoding='latin1')


                output_flip_smpl = smpl(
                    body_pose=thetas[:, 3:],
                    global_orient=thetas[:, :3],
                    pose2rot=True,
                    create_transl=True,
                    transl = all_translations_tensor
                )

                verts_all = output_flip_smpl.vertices.detach()

                verts_all = verts_all.cpu().numpy()
                output_folder = dirpath  # Save in the same directory


                #render_and_save(verts_all, osp.join(output_folder, 'mesh.mp4'), keep_imgs=False, fps=fps_in, draw_face=True)

                translations = all_translations_tensor.cpu().numpy()

                thetas = thetas.cpu().numpy()

                # # Define the rotation matrix
                # R_x = np.array([
                #     [1,  0,  0],
                #     [0, -1,  0],
                #     [0,  0, -1]
                # ])

                # # Assuming verts_all is already defined with shape (1262, 6890, 3)
                # # Apply the rotation
                # verts_all = np.einsum('ij,klj->kli', R_x, verts_all)

                #Save outputs in the same directory as the data file
                
                save_smpl(translations, thetas, verts_all, output_folder)

# Root folder containing all test sets
root_folder = base_pred_dir

In [40]:
# Take all motionbert files and calculate all SMPL parameters (+ vertices)

def process_motionbert_files(root_path):
    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if file == 'X3D.npy':
                # Full path to the data file
                data_path = os.path.join(dirpath, file)

                # Call calculate_smpl_output
                data = np.load(data_path)
                translations, thetas = calculate_smpl_output(data)


                thetas = torch.tensor(thetas).to('cuda')

                all_translations_tensor = torch.tensor(translations, dtype=torch.double).to('cuda')

                path_split_list = dirpath.split(os.sep)
                seq_name =  path_split_list[-3]
                #print('SEQ name is:', seq_name)
                person_nr = int(path_split_list[-1][-1]) 

                # campose_valid = get_campose_valid(seq_name, person_nr)
                # #print(len(campose_valid))

                # all_translations_tensor = all_translations_tensor[campose_valid]
                # thetas = thetas[campose_valid]


                output_flip_smpl = smpl(
                    body_pose=thetas[:, 3:],
                    global_orient=thetas[:, :3],
                    pose2rot=True,
                    create_transl=True,
                    transl = all_translations_tensor
                )

                verts_all = output_flip_smpl.vertices.detach()

                verts_all = verts_all.cpu().numpy()
                output_folder = dirpath  # Save in the same directory


                #render_and_save(verts_all, osp.join(output_folder, 'mesh.mp4'), keep_imgs=False, fps=fps_in, draw_face=True)

                translations = all_translations_tensor.cpu().numpy()

                thetas = thetas.cpu().numpy()

                # Define the rotation matrix
                R_x = np.array([
                    [1,  0,  0],
                    [0, -1,  0],
                    [0,  0, -1]
                ])

                # Assuming verts_all is already defined with shape (1262, 6890, 3)
                # Apply the rotation
                verts_all = np.einsum('ij,klj->kli', R_x, verts_all)

                #Save outputs in the same directory as the data file
                
                save_smpl(translations, thetas, verts_all, output_folder)

# Root folder containing all test sets
root_folder = base_pred_dir

In [41]:
process_motionbert_files(root_folder)

In [3]:
import numpy as np

In [65]:
res = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\all_thetas.npy")

In [66]:
res.shape

(895, 72)

In [103]:
# RENDER AND LOOK WHAT TF IS WRONG Nl

In [76]:
file_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\vertices.npy"
vertices = np.load(file_path)
file_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\vertices_all_frames.npy"
vertices_all_frames = np.load(file_path)

In [81]:
vertices 

array([[[0.50774313, 0.47102964, 0.21845487],
        [0.49936341, 0.45677437, 0.2236435 ],
        [0.51150523, 0.45228129, 0.21643265],
        ...,
        [0.40190039, 0.44389135, 0.09318051],
        [0.40134032, 0.44518407, 0.09518934],
        [0.3981391 , 0.44257733, 0.09703511]],

       [[0.50848683, 0.47181244, 0.21798028],
        [0.50018968, 0.45753138, 0.22317786],
        [0.5123067 , 0.45309245, 0.21590144],
        ...,
        [0.40211151, 0.44448478, 0.09316706],
        [0.40155601, 0.44577074, 0.09518098],
        [0.39837849, 0.44315236, 0.09703512]],

       [[0.51024218, 0.47144827, 0.21662677],
        [0.50202228, 0.45714374, 0.2218423 ],
        [0.5140904 , 0.45274851, 0.2144682 ],
        ...,
        [0.40295759, 0.44413438, 0.09252538],
        [0.40241282, 0.44541567, 0.09454517],
        [0.39925829, 0.4427856 , 0.09641637]],

       ...,

       [[0.26953391, 0.49526304, 0.28865103],
        [0.26386925, 0.48089427, 0.29595449],
        [0.27578705, 0

In [77]:
vertices.shape

(895, 6890, 3)

In [69]:
file_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_1\vertices_all_frames.npy"

In [26]:
vertices_all_frames = np.load(file_path)

In [48]:
vertices_all_frames.shape

(895, 6890, 3)

In [47]:
render_and_save(vertices_all_frames[0:200], osp.join(r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\flat_guitar_01\test', 'mesh.mp4'), keep_imgs=False, fps=fps_in, draw_face=True)

100%|██████████| 200/200 [02:57<00:00,  1.13it/s]


In [83]:
render_and_save(vertices[0:200], osp.join(r'E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\flat_guitar_01\test', 'mesh2.mp4'), keep_imgs=False, fps=fps_in, draw_face=True)

100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


In [99]:
# SOLUTION: USE OLD WAY OF EXTRACTING THE VERTICES

In [97]:
file_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\vertices.npy"
vertices = np.load(file_path)
file_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert\person_2\vertices_all_frames.npy"
vertices_all_frames = np.load(file_path)

In [98]:
vertices.shape

(895, 6890, 3)

In [ ]:
# NOTE TO ME: TRY TWO THINGS: FILTER VERTICES EXTRACTED HERE BY COMPOSE VALID (look at linux extracting) OR: USE OLD, OTHER WAY OF EXTRACTING THE VERTICES